# Initialization (RUN ALL BELLOW)

In [ ]:
pip install fmpsdk

In [9]:
import pandas as pd
from urllib.request import urlopen
import certifi
import ssl
import json
import fmpsdk
from dateutil.relativedelta import relativedelta
from openpyxl.utils import get_column_letter
#from google.colab import files
from tqdm import tqdm

# Create a secure SSL context
context_init = ssl.create_default_context(cafile=certifi.where())





,name,ticker,Bloomberg ticker,S&P rating,Moody rating,country,cik,isin,cusip,Sector,-,Industry
0,Walmart Inc.,WMT,WMT US equity,AA,Aa2,US,104169.0,US9311421039,931142103,Consumer Defensive,Discount Stores,Discount Stores
1,"Amazon.com, Inc.",AMZN,AMZN US equity,AA,A1,US,1018724.0,US0231351067,023135106,Consumer Cyclical,Internet Retail,Specialty Retail
2,NaN,386,386 HK equity,A+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple Inc.,AAPL,AAPL US equity,AA+,Aaa,US,320193.0,US0378331005,037833100,Technology,Consumer Electronics,Consumer Electronics
4,CVS Health Corporation,CVS,CVS US equity,BBB,Baa2,US,64803.0,US1266501006,126650100,Healthcare,Healthcare Plans,Medical - Healthcare Plans
...,...,...,...,...,...,...,...,...,...,...,...,...
1433,"NetApp, Inc.",NTAP,NTAP US equity,BBB+,Baa2,US,1002047.0,US64110D1046,64110D104,Technology,Computer Hardware,Computer Hardware
1434,Edison International,EIX,EIX US equity,BBB,Baa2,US,827052.0,US2810201077,281020107,Utilities,Utilities—Regulated Electric,Regulated Electric
1435,Cvent Holding Corp.,CVT,CVT US equity,NaN,NaN,US,1827075.0,US1266771039,126677103,Technology,Software—Application,Software - Application
1436,"Sun Communities, Inc.",SUI,SUI US equity,BBB,NaN,US,912593.0,US8666741041,866674104,Real Estate,REIT—Residential,REIT - Residential


Data extraction functions:

In [ ]:
def get_data_url(url):
    response = urlopen(url, context=context_init)
    data = response.read().decode("utf-8")
    data=json.loads(data)
    return pd.DataFrame(data)

def get_org_chart(ticker):
    apikey='2723105eaade3c1ca8d66ca5c567b590'
    url = (f'https://financialmodelingprep.com/api/v3/key-executives/{ticker}?apikey={apikey}')
    df_=get_data_url(url)
    df_=df_[['title']+['name']]
    df_=df_.drop_duplicates(subset='name')
    #df_=df_.drop_duplicates(subset='title', inplace=True)
    return df_

def stock_price(ticker):
    try:
        stock_prices = fmpsdk.historical_price_full(apikey='2723105eaade3c1ca8d66ca5c567b590', symbol=ticker)
        closing_prices = [price['close'] for price in stock_prices]

        weekly_stock_variation = (closing_prices[0]-closing_prices[7])/closing_prices[7]

        if weekly_stock_variation >=0:
            return "+"+str(round(weekly_stock_variation*100,2))+"%"
        else:
            return str(round(weekly_stock_variation*100,2))+"%"

    except Exception as e:
        print(f"Unexpected error: {e}")
        return "FAIL"


def data_inventory(ticker,period):

    apikey='2723105eaade3c1ca8d66ca5c567b590'

    symbol: str = ticker

    try:
        data_BSS=pd.DataFrame(fmpsdk.balance_sheet_statement(apikey=apikey, symbol=symbol, period=period))
        #return '$'+str((data_BSS['inventory'].iloc[0])/1000000)+"M"
        return data_BSS['inventory'].iloc[0]
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "FAIL"


def get_latest_earnings_transcript(ticker):
    apikey = '2723105eaade3c1ca8d66ca5c567b590'

    url = f'https://financialmodelingprep.com/api/v3/earning_call_transcript/{ticker}?apikey={apikey}'

    try:
        data = get_data_url(url)
        date_latest=pd.to_datetime(data["date"].iloc[0])
        date_next=date_latest + relativedelta(months=3)
        transcript=data['content'].iloc[0]
        return transcript, date_next
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "FAIL"

def return_company_profile(tck):
    # Get company profile and industry from the ticker
    company_info = fmpsdk.company_profile(symbol=tck, apikey='2723105eaade3c1ca8d66ca5c567b590')
    try:
        return pd.DataFrame([company_info[0]])
    except Exception as e:
        print(f"Unexpected error: {e}")
        return pd.DataFrame([{'companyName': 'FAIL', 'industry': 'FAIL'}])

def new_executive(tck):
    return 'in dev.'

def credit_rating(tck):
    
    file_2023='https://raw.githubusercontent.com/alexandre-tcc/target-client_tracking/refs/heads/main/TCC%20companies%20mapping%2010142024.csv'
    file_2025='https://raw.githubusercontent.com/alexandre-tcc/target-client_tracking/refs/heads/main/TCC%20Pre-qualified%20Targets%20202504421.csv'
    
    df_2023=pd.read_csv(file_2023)
    df_2025=pd.read_csv(file_2025)
    
    df_rating_filtered_2025=df_2025[df_2025['ticker']==tck]

    if df_rating_filtered_2025.empty:
        df_rating_filtered_2023=df_2023[df_2023['ticker']==tck]
        if df_rating_filtered_2023.empty:
            return 'Need Manual research'
        else:
            return df_rating_filtered_2023['S&P rating'].iloc[0]
    else:
        return df_rating_filtered_2025['S&P rating'].iloc[0]

Output functions

In [ ]:
def generate_table(lst_tck):
    df_rtr = pd.DataFrame(columns=['Company Name', 'Ticker', 'Industry', 'Stock Trend (7days)', 'Inventory ($)', 'S&P Rating (2023)','Next Earnings', 'Latest Earnings Presentation','Exec. Update'])
    for tck in tqdm(lst_tck, desc="Progress", unit="ticker"):
        company_profil=return_company_profile(tck)
        data_earning_=get_latest_earnings_transcript(tck)

        stock_trend=stock_price(tck)
        next_call=data_earning_[1]
        transcript=data_earning_[0]
        inventory=data_inventory(tck, 'annual')
        company_name=company_profil['companyName'].iloc[0]
        company_industry=company_profil['industry'].iloc[0]
        credit_SnP=credit_rating(tck)
        exec_update=new_executive(tck)

        df_rtr.loc[len(df_rtr)]=[company_name, tck, company_industry, stock_trend, inventory, credit_SnP, next_call, transcript, exec_update]

    #df_rtr.to_excel('Target List Intelligence.xlsx', index=False)
    return df_rtr


def generate_output(list_tck):
    df=generate_table(list_tck)
    with pd.ExcelWriter('Target List Intelligence.xlsx', engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Target List')
        worksheet = writer.sheets['Target List']

        for i in range(len(df.columns)):
            col_letter = get_column_letter(i + 1)  # 1-based indexing
            worksheet.column_dimensions[col_letter].width = 25  # or any width you want
    files.download('Target List Intelligence.xlsx')

# **Run Program**
## You can add/remove tickers from this list
## Generated file will automatically be downloaded

In [ ]:
tickers = ['MSFT', 'TSLA', 'AAPL', 'GOOG', 'AMZN', 'META', 'NVDA', 'BRK.B', 'JPM', 'UNH']

generate_output(tickers)

# Git COMMIT (DO NOT RUN)

In [10]:
!git add .


In [11]:
!git commit -m "new columns to the DF"


[main 52d8050] new columns to the DF
 3 files changed, 1222 insertions(+), 613 deletions(-)
 create mode 100644 TCC Pre-qualified Targets 202504421.csv


In [12]:
!git push origin main


To https://github.com/alexandre-tcc/target-client_tracking.git
   3111022..52d8050  main -> main
